In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 97.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 101.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 21.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from transformers import AutoTokenizer

checkpoint = "pranaydeeps/Ancient-Greek-BERT"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/506 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/530k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/800k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset

# dataset_nt = load_dataset("csv", data_files='/content/drive/MyDrive/Datasets/00-new-testament.csv')
dataset_lxx = load_dataset("csv", data_files='/content/drive/MyDrive/Datasets/lxx_with_genres.csv')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-0ad455a0b222f6f1/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from datasets import DatasetDict

dataset_lxx = dataset_lxx["train"].train_test_split()
test_valid = dataset_lxx["train"].train_test_split()
full_dataset_lxx = DatasetDict({
    'train': dataset_lxx['train'],
    'test': test_valid['train'],
    'valid': test_valid['test']})

In [ ]:
book_list = list(set(dataset_lxx["train"]["Book"] + dataset_lxx["test"]["Book"]))

def get_label_int(book):
  for x in range(0, len(book_list)):
    if(book == book_list[x]):
      return x

def tokenization(raw):
  result = tokenizer(raw["Verse Text"], truncation=True)
  result["label"] = [get_label_int(book) for book in raw["Book"]]
  return result

tokenized_train = full_dataset_lxx["train"].map(tokenization, batched=True)
tokenized_test = full_dataset_lxx["test"].map(tokenization, batched=True)
tokenized_valid = full_dataset_lxx["valid"].map(tokenization, batched=True)

tokenized_dataset_lxx = DatasetDict({
    'train': tokenized_train,
    'test': tokenized_test,
    'valid': tokenized_valid})

  0%|          | 0/23 [00:00<?, ?ba/s]

  0%|          | 0/18 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

In [ ]:
from transformers import DataCollatorWithPadding, TrainingArguments, AutoModelForSequenceClassification

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=57)

Downloading:   0%|          | 0.00/452M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at pranaydeeps/Ancient-Greek-BERT and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset_lxx["train"],
    eval_dataset=tokenized_dataset_lxx["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: RunningNumber: Book: Chapter, Verse Text, Book, Verse Number, Chapter, Genre. If RunningNumber: Book: Chapter, Verse Text, Book, Verse Number, Chapter, Genre are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 22926
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 8598
  Number of trainable parameters = 1129651

Epoch,Training Loss,Validation Loss,Accuracy
1,1.867200,1.294957,0.611305
2,1.139500,0.648529,0.802163
3,0.608300,0.373625,0.885206


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test_trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in test_trainer/checkpoint-500/special_tokens_map.json
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test_trainer/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test_trainer/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to test_trainer/checkpoint-1500
Configuration saved in test_trainer/checkpoint-1500/config.json
Model weights saved in test_trainer/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in test_trainer/checkpoint-1500/tokenizer_config.json
Special tokens file saved

TrainOutput(global_step=8598, training_loss=1.358876649599681, metrics={'train_runtime': 1279.7361, 'train_samples_per_second': 53.744, 'train_steps_per_second': 6.719, 'total_flos': 2236159164429900.0, 'train_loss': 1.358876649599681, 'epoch': 3.0})

In [ ]:
dataset_nt = load_dataset("csv", data_files='/content/drive/MyDrive/Datasets/00-new-testament.csv')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-aa1673994b0ab2ef/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from transformers import pipeline
import re

def book_predicted(text):
  classif = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)
  result = classif(text)
  label = int(re.findall("\d+$", result[0]["label"])[0])
  return book_list[label]

#text = "καὶ ὡμίλησεν αὐτοῖς ὁ βασιλεύς, καὶ οὐχ εὑρέθη ἐν τοῖς σοφοῖς ὅμοιος τῷ Δανιηλ καὶ Ανανια καὶ Μισαηλ καὶ Αζαρια"
#print(book_predicted(text))

In [ ]:
dataset_nt_predicted = dataset_nt["train"]
dataset_nt_predicted = dataset_nt_predicted.map(lambda x: {"prediction" : book_predicted(x["text"])})
dataset_nt_predicted

  0%|          | 0/7956 [00:00<?, ?ex/s]

Dataset({
    features: ['book', 'chapter', 'verse', 'text', 'prediction'],
    num_rows: 7956
})

In [ ]:
import pandas as pd

df_nt = dataset_nt_predicted.to_pandas()

In [ ]:
df_result = df_nt.groupby("book")["prediction"].value_counts()

In [ ]:
df_result["hebrews"]

prediction
ΣΟΦΙΑ ΣΑΛΩΜΩΝΟΣ                              78
ΜΑΚΚΑΒΑΙΩΝ Β                                 38
ΜΑΚΚΑΒΑΙΩΝ Δ                                 28
ΗΣΑΙΑΣ                                       21
ΕΣΔΡΑΣ Α (liber apocryphus)                  19
ΠΡΟΛΟΓΟΣ                                     19
ΜΑΚΚΑΒΑΙΩΝ Γ                                 12
ΓΕΝΕΣΙΣ                                      11
ΜΑΚΚΑΒΑΙΩΝ Α                                 11
ΨΑΛΜΟΙ                                       10
ΕΞΟΔΟΣ                                        7
ΠΑΡΟΙΜΙΑΙ                                     6
ΙΕΡΕΜΙΑΣ                                      5
ΙΗΣΟΥΣ                                        5
ΙΩΒ                                           4
ΤΩΒΙΤ (Codices Alexandrinus et Vaticanus)     4
ΩΔΑΙ                                          4
ΕΠΙΣΤΟΛΗ ΙΕΡΕΜΙΟΥ                             3
ΑΡΙΘΜΟΙ                                       2
ΔΕΥΤΕΡΟΝΟΜΙΟΝ                                 2
ΙΕΖΕΚΙΗΛ                     

In [ ]:
metric = evaluate.load("f1")

def compute_metrics_f1(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")

In [ ]:
trainer_f1 = Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset_lxx["train"],
    eval_dataset=tokenized_dataset_lxx["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_f1,
)

In [ ]:
trainer_f1.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Verse Number, Book, Chapter, RunningNumber: Book: Chapter, Genre, Verse Text. If Verse Number, Book, Chapter, RunningNumber: Book: Chapter, Genre, Verse Text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 17194
  Batch size = 8


{'eval_loss': 0.39896127581596375,
 'eval_f1': 0.87078102320949,
 'eval_runtime': 67.3009,
 'eval_samples_per_second': 255.479,
 'eval_steps_per_second': 31.946}